In [61]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset,Dataset
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_curve
#Class to load the dataset images from drivce
import os
import cv2
import numpy as np

from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import keras


In [62]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [63]:
batch_size = 32  # for updation of weights in a batch.
num_classes = 3  # Number of class for the dataset
epochs = 100

In [64]:

class SimpleDatasetLoader:
    # Method: Constructor
    def __init__(self, preprocessors=None):
        """
        :param preprocessors: List of image preprocessors
        """
        self.preprocessors = preprocessors

        if self.preprocessors is None:
            self.preprocessors = []

    # Method: Used to load a list of images for pre-processing
    def load(self, image_paths, verbose=-1):
        """
        :param image_paths: List of image paths
        :param verbose: Parameter for printing information to console
        :return: Tuple of data and labels
        """
        data, labels = [], []

        for i, image_path in enumerate(image_paths):
            image = cv2.imread(image_path)
            label = image_path.split(os.path.sep)[-2]

            if self.preprocessors is not None:
                for p in self.preprocessors:
                    image = p.preprocess(image)

            data.append(image)
            labels.append(label)

            if verbose > 0 and i > 0 and (i+1) % verbose == 0:
                print('[INFO]: Processed {}/{}'.format(i+1, len(image_paths)))

        return (np.array(data), np.array(labels))

In [65]:
#Class Preprocessror 
class SimplePreprocessor:
    # Method: Constructor
    def __init__(self, width, height, interpolation=cv2.INTER_AREA):
        """
        :param width: Image width
        :param height: Image height
        :param interpolation: Interpolation algorithm
        """
        self.width = width
        self.height = height
        self.interpolation = interpolation

    # Method: Used to resize the image to a fixed size (ignoring the aspect ratio)
    def preprocess(self, image):
        """
        :param image: Image
        :return: Re-sized image
        """
        return cv2.resize(image, (self.width, self.height), interpolation=self.interpolation)

In [66]:
#Class Preprocessror 
class SimplePreprocessor:
    # Method: Constructor
    def __init__(self, width, height, interpolation=cv2.INTER_AREA):
        """
        :param width: Image width
        :param height: Image height
        :param interpolation: Interpolation algorithm
        """
        self.width = width
        self.height = height
        self.interpolation = interpolation

    # Method: Used to resize the image to a fixed size (ignoring the aspect ratio)
    def preprocess(self, image):
        """
        :param image: Image
        :return: Re-sized image
        """
        return cv2.resize(image, (self.width, self.height), interpolation=self.interpolation)

In [67]:
image_paths = list(paths.list_images("animals"))

# Initialize SimplePreprocessor and SimpleDatasetLoader and load data and labels
print('[INFO]: Images loading....')
sp = SimplePreprocessor(32, 32)
sdl = SimpleDatasetLoader(preprocessors=[sp])
(data, labels) = sdl.load(image_paths, verbose=500)

# Reshape from (3000, 32, 32, 3) to (3000, 32*32*3=3072)
data = data.reshape((data.shape[0], 3072))
print(data.shape)
# Print information about memory consumption
# print('[INFO]: Features Matrix: {:.1f}MB'.format(float(data.nbytes / 1024*1000.0)))

# Encode labels as integers
le = LabelEncoder()
labels = le.fit_transform(labels)
# Split data into training (75%) and testing (25%) data
X_train, X_test, y_train, y_test= train_test_split(data, labels, test_size=0.25, random_state=42)

[INFO]: Images loading....
[INFO]: Processed 500/3000
[INFO]: Processed 1000/3000
[INFO]: Processed 1500/3000
[INFO]: Processed 2000/3000
[INFO]: Processed 2500/3000
[INFO]: Processed 3000/3000
(3000, 3072)


In [68]:
print(X_train.shape)
print('-------------')
print(X_test.shape)
print('--------------')
print(y_train.shape)
print('--------------')
print(y_test.shape)

(2250, 3072)
-------------
(750, 3072)
--------------
(2250,)
--------------
(750,)


In [69]:
le = preprocessing.LabelEncoder()
y_test = le.fit_transform(y_test)

#reshaping data for convolutional neural netowrks
X_train = X_train.reshape(2250,32,32,3) 
y_train = y_train.reshape(2250,1)
X_test = X_test.reshape(750,32,32,3)
y_test = y_test.reshape(750,1)

In [70]:
#converting data to float so we can divide by 255(UNICODE)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [71]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [72]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',input_shape=X_train.shape[1:])) 
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) #getting max values from matrices
model.add(Dropout(0.25)) #dropping weights

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) #getting max values from matrices
model.add(Dropout(0.25))

model.add(Flatten()) #flattening data
model.add(Dense(512)) # connecting neurons deeply
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [73]:
#decay = regularization
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [74]:
model.fit(X_train, y_train,batch_size=batch_size,epochs=epochs,validation_data=(X_test, y_test),shuffle=True)

Epoch 1/100
71/71 [==============================] - 10s 129ms/step - loss: 1.0509 - accuracy: 0.4311 - val_loss: 0.9769 - val_accuracy: 0.5707
Epoch 2/100
71/71 [==============================] - 9s 120ms/step - loss: 0.9319 - accuracy: 0.5227 - val_loss: 0.9144 - val_accuracy: 0.5213
Epoch 3/100
71/71 [==============================] - 9s 128ms/step - loss: 0.8498 - accuracy: 0.5702 - val_loss: 0.7962 - val_accuracy: 0.5907
Epoch 4/100
71/71 [==============================] - 9s 122ms/step - loss: 0.7937 - accuracy: 0.5862 - val_loss: 0.8015 - val_accuracy: 0.5600
Epoch 5/100
71/71 [==============================] - 9s 128ms/step - loss: 0.7731 - accuracy: 0.6107 - val_loss: 0.7592 - val_accuracy: 0.6187
Epoch 6/100
71/71 [==============================] - 9s 130ms/step - loss: 0.7486 - accuracy: 0.6142 - val_loss: 0.8311 - val_accuracy: 0.5600
Epoch 7/100
71/71 [==============================] - 11s 149ms/step - loss: 0.7222 - accuracy: 0.6333 - val_loss: 0.7680 - val_accuracy: 0.60

Epoch 58/100
71/71 [==============================] - 9s 132ms/step - loss: 0.3963 - accuracy: 0.8253 - val_loss: 0.6158 - val_accuracy: 0.7333
Epoch 59/100
71/71 [==============================] - 10s 141ms/step - loss: 0.3943 - accuracy: 0.8347 - val_loss: 0.7387 - val_accuracy: 0.7107
Epoch 60/100
71/71 [==============================] - 10s 145ms/step - loss: 0.3847 - accuracy: 0.8316 - val_loss: 0.7097 - val_accuracy: 0.7067
Epoch 61/100
71/71 [==============================] - 10s 144ms/step - loss: 0.3845 - accuracy: 0.8311 - val_loss: 0.6386 - val_accuracy: 0.7267
Epoch 62/100
71/71 [==============================] - 9s 132ms/step - loss: 0.3767 - accuracy: 0.8440 - val_loss: 0.7179 - val_accuracy: 0.6827
Epoch 63/100
71/71 [==============================] - 10s 136ms/step - loss: 0.3811 - accuracy: 0.8356 - val_loss: 0.6223 - val_accuracy: 0.7360
Epoch 64/100
71/71 [==============================] - 10s 136ms/step - loss: 0.3753 - accuracy: 0.8387 - val_loss: 0.6983 - val_accu

In [75]:
#evaluating 
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

24/24 [==============================] - 1s 25ms/step - loss: 0.7708 - accuracy: 0.7333
Test loss: 0.7708353400230408
Test accuracy: 0.7333333492279053


In [76]:
#predicting
pred = model.predict(X_test)

24/24 [==============================] - 1s 25ms/step


In [77]:
#making predictions 1D for accuracy and report
pred = np.argmax(pred, axis=1)

In [78]:
#making testing labels 1D for accuracy and report
y_test = np.argmax(y_test,axis=1)

In [79]:
#accuracy
accuracy = accuracy_score(y_test, pred)

In [80]:
accuracy

0.7333333333333333

In [81]:
from sklearn.metrics import classification_report

report = classification_report(y_test, pred)

In [82]:
print(report)

              precision    recall  f1-score   support

           0       0.77      0.51      0.61       262
           1       0.59      0.83      0.69       249
           2       0.94      0.88      0.91       239

    accuracy                           0.73       750
   macro avg       0.76      0.74      0.74       750
weighted avg       0.76      0.73      0.73       750

